In [2]:
"""Tutorial: Overlap, Kinetic, and Dipole Integrals"""

__author__    = ["Justin M. Turney"]
__credit__    = ["Justin M. Turney"]

__copyright__ = "(c) 2014-2017, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-08-10"

# NOTE
*** This Tutorial is a work in progress. The code is present, but the explanations still require further review and edits***

# Overlap, Kinetic, and Dipole Integrals

In this tutorial we will compute the overlap and kinetic energy integrals encountered in Hartree-Fock. We will also compute the dipole integrals to obtain the molecular dipole moment. 

## Recurrence formula for one-electron integrals over Gaussian functions
Direct calculation of all integrals is not only complicated, but expensive. Here, we will implement an Obara-Saika recursion scheme for a simpler and more efficient implementation. For example, for the overlap integrals, given the value of $(s|s)$, one can recursively determine all other overlap integrals.

### Cartesian Gaussian functions
Denote the origin of a 3-dimensional cartesian gaussian function by the coordinates $\mathbf{R} = (R_x, R_y, R_z)$.
Let $\mathbf{r} = (x, y, z)$ be the coordinates of the electron, and $\alpha$ be the orbital exponent. We can now define an *unnormalized* cartesian gaussian function as:

\begin{equation}
\phi(\mathbf r; \alpha, \mathbf n, \mathbf R) = (x - R_x)^{n_x} (y - R_y)^{n_y} (z - R_z)^{n_z} \exp[-\alpha (\mathbf r - \mathbf R)^2]
\end{equation}

where $\alpha$ is the orbital exponent, and $\mathbf{n} = (n_x, n_y, n_z)$ is the angular momentum index vector. The sum $n_x + n_y + n_z = \lambda$ will hereafter be referred to as the **angular momentum**. We define a **shell** to be a set of functions (*components*) which share the same origin $\mathbf{R}$, angular momentum $\lambda$, and orbital exponent $\alpha$. 

The shells with $\lambda$  equal to $0, 1, 2,...,$ are referred to as the $s, p, d, ...$ shell. Each shell has $(\lambda + 1) (\lambda + 2)/2$ components. The $s$ shell, with angular momentum $\lambda = 0$ has one component usually designated as $s$. The $p$ shell ($\lambda = 1$) has three components, designated as $p_x, p_y, p_z$. The $d$ shell ($\lambda = 2$) has six components, designated as $d_{xx}, d_{yy}, d_{zz}, d_{xy}, d_{xz}, d_{yz}$.

In quantum chemistry, we typically represent a single component (an **atomic orbital**) by a linear combination of gaussians, hereon referred to as **primitives**. For example, the STO-3G basis set uses three primitives for each atomic orbital basis function.

Using our angular momentum index vector $\mathbf{n}$ we denote the single component of the $s$ shell to have angular momentum index $\mathbf{n} = \mathbf{0} = (0, 0, 0)$. Since the $p$ shell has three components, we may compactly express the angular momentum index vector as $\mathbf{1}_i$ where $i$ may be $x$, $y$, or $z$, and $\mathbf{1}_i = (\delta_{ix}, \delta_{iy}, \delta_{iz})$. For example, $p_x$ may be represented as $\mathbf{1}_x = (1, 0, 0)$. For the six components of the $d$ shell, we require a sum of two angular momentum index vectors $\mathbf{1}_i + \mathbf{1}_j$, where $(i,j = x,y,z)$. In this notation, the $d_{xy}$ component is $\mathbf{1}_x + \mathbf{1}_y = (1,0,0) + (0,1,0) = (1,1,0)$. To obtain higher order angular momentum components, we add the appropriate number of $\mathbf{1}_i$'s ($\mathbf{n}$'s)


### Two-center overlap integrals
Two-center overlap integrals over unnormalized cartesian gaussian functions are of the form:
\begin{equation}
(\mathbf a|\mathbf b) = \int d\mathbf r\ \phi(\mathbf r; \alpha_a, \mathbf a, \mathbf A)\phi(\mathbf r; \alpha_b, \mathbf b, \mathbf B)
\end{equation}

given $(\mathbf 0_A | \mathbf 0_B)$, we can use the Obara-Saika recursion relation to obtain overlap integrals between all basis functions. The overlap over $s$ functions is given by:

\begin{equation}
(\mathbf 0_A | \mathbf 0_B) = \left(\frac{\pi}{\alpha}\right)^{3/2} \exp[-\xi(\mathbf A-\mathbf B)^2]
\end{equation}

where $\alpha = \alpha_a + \alpha_b$ and $\zeta = \frac{\alpha_a\alpha_b}{\alpha}$.
The recursion relations are given below. For a full derivation, see the appendix, or the original paper by Obara and Saika (CITE). To increment the left side angular momentum:

\begin{equation}
(\mathbf a+\mathbf 1_i|\mathbf b) = (\mathbf{P - A})(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}
and similarily, to increment the right side:
\begin{equation}
(\mathbf a|\mathbf b+\mathbf 1_i) = (\mathbf{P - B})(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}


where \begin{equation}\mathbf{P} = \frac{\alpha_a \mathbf{A} + \alpha_b \mathbf{B}} {\alpha} \end{equation}
and ${N}_i$ is ???



### Write the recursion function

The Obara-Saika recursion relationships depend on $\mathbf{P-A}$ and  $\mathbf{P-B}$, $\alpha$,  and two angular momenta (for $\mathbf a$ and $\mathbf b$) values which we will denote as `PA`, `PB`, `alpha`, `AMa`, and `AMb`. 
Let's write a function that takes these parameters and returns three matrices containing the x, y and z components of our unnormalized overlap integrals. These same components also can be used to construct our kinetic energy and dipole integrals later, as we will see.

For now, we will set $(\mathbf 0 | \mathbf 0)$ to $1.0$ and account for it later. 

In [1]:
import psi4
import numpy as np

def os_recursion(PA, PB, alpha, AMa, AMb):
    if len(PA) != 3 or len(PB) != 3:
        raise "PA and PB must be xyz coordinates."

    # Allocate space x, y, and z matrices
    # We add one because the equation for the kinetic energy
    # integrals require terms one beyond those in the overlap
    x = np.zeros((AMa + 1, AMb + 1))
    y = np.zeros((AMa + 1, AMb + 1))
    z = np.zeros((AMa + 1, AMb + 1))

    # Define 1/2alpha factor for convenience
    oo2a = 1.0 / (2.0 * alpha)

    # Set initial conditions (a|b) to 'start' for each component
    x[0, 0] = y[0, 0] = z[0, 0] = 1.0

    
    # BEGIN RECURSION
    # Upward recursion in b for a = 0
    # Fill in the [0,1] position with PB
    if AMb > 0:
        x[0, 1] = PB[0]
        y[0, 1] = PB[1]
        z[0, 1] = PB[2]

    # Fill in the rest of row zero
    for b in range(1, AMb):
        x[0, b + 1] = PB[0] * x[0, b] + b * oo2a * x[0, b - 1]
        y[0, b + 1] = PB[1] * y[0, b] + b * oo2a * y[0, b - 1]
        z[0, b + 1] = PB[2] * z[0, b] + b * oo2a * z[0, b - 1]

    # Upward recursion in a for all b's
    # Fill in the [1,0] position with PA
    if AMa > 0:
        x[1, 0] = PA[0]
        y[1, 0] = PA[1]
        z[1, 0] = PA[2]

        # Fill in the rest of row one
        for b in range(1, AMb + 1):
            x[1, b] = PA[0] * x[0, b] + b * oo2a * x[0, b - 1]
            y[1, b] = PA[1] * y[0, b] + b * oo2a * y[0, b - 1]
            z[1, b] = PA[2] * z[0, b] + b * oo2a * z[0, b - 1]

        # Fill in the rest of column 0
        for a in range(1, AMa):
            x[a + 1, 0] = PA[0] * x[a, 0] + a * oo2a * x[a - 1, 0]
            y[a + 1, 0] = PA[1] * y[a, 0] + a * oo2a * y[a - 1, 0]
            z[a + 1, 0] = PA[2] * z[a, 0] + a * oo2a * z[a - 1, 0]
    
        # Fill in the rest of the a'th row
            for b in range(1, AMb + 1):
                x[a + 1, b] = PA[0] * x[a, b] + a * oo2a * x[a - 1, b] + b * oo2a * x[a, b - 1]
                y[a + 1, b] = PA[1] * y[a, b] + a * oo2a * y[a - 1, b] + b * oo2a * y[a, b - 1]
                z[a + 1, b] = PA[2] * z[a, b] + a * oo2a * z[a - 1, b] + b * oo2a * z[a, b - 1]

    # Return the results
    return (x, y, z)


# Overlap Integrals

Now that we have our recursion set up, we are ready to compute the integrals, starting with the overlap integrals. First we need a molecule and a basis set. We compute a quick Hartree-Fock to obtain a wavefunction for easy access to the basis set. Later, we can compare Psi4's HF energy to the energy derived from our hand-coded integrals.

In the Psi4 options, we set `puream` to be off. By default, Psi4 uses spherical harmonic gaussians, sometimes referred to as "pure angular momentum", whereas we will be coding with Cartesian Gaussians.

In [5]:
mol = psi4.geometry("""
                        O
                        H 1 1.1
                        H 1 1.1 2 104
                        symmetry c1
                        """)


psi4.set_options({'basis':        'sto-3g',
                  'scf_type':         'pk',
                  'mp2_type':       'conv',
                  'puream':              0,
                  'e_convergence':    1e-8,
                  'd_convergence':    1e-8})

scf_e, scf_wfn = psi4.energy('scf', return_wfn = True)

basis = scf_wfn.basisset()



The factored form of the two-center overlap integrals is
\begin{equation}
(\mathbf a | \mathbf b) = \exp[-\zeta(\mathbf A-\mathbf B)^2]\ I_x(n_{ax},n_{bx},n_{cx})\ I_y(n_{ay},n_{by},n_{cy})\ I_z(n_{az},n_{bz},n_{cz})
\end{equation}

where $I_x$ is the x component of the unnormalized overlap matrix.

To obtain the overlap matrix, we must loop over all shells, and for each shell, loop over the primitive gaussians.

~~~python
# make space to store the overlap integral matrix
S = np.zeros((basis.nao(),basis.nao()))

# loop over the shells, basis.nshell is the number of shells
for i in range(basis.nshell()):
    for j in range(basis.nshell()):
        #basis.shell is a shell (1s, 2s, 2p, etc.)
        #for water, there are 5 shells: (H: 1s, H: 1s, O: 1s, 2s, 2p)
        ishell = basis.shell(i) 
        jshell = basis.shell(j)
        #each shell has some number of primitives which make up each component of a shell
        #sto-3g has 3 primitives for every component of every shell.
        nprimi = ishell.nprimitive 
        nprimj = jshell.nprimitive
       
~~~

At this point we access the information for each primitive basis set $\mathbf{a}$ and $\mathbf{b}$ in $(\mathbf{a}|\mathbf{b})$. The `basis.shell()` (which we have set to `ishell` and `jshell`) is a `psi4.core.GaussianShell` object, which contains all the information we need about the primitives within a shell, such as the exponent `(shell().exp())`, the coefficient `(shell().coef())`, the angular momentum `(shell().am)` and which atom number it is centered over `(shell().ncenter)` which we can use to get the coordinates of the primitive center ($\mathbf{A}$ or $\mathbf{B}$) (*whew!*). We store all of these parameters as variables in our for-loops for easy access. Continuing our sample script:

~~~python
        #loop over the primitives within a shell
        for a in range(nprimi):  
            for b in range(nprimj):
                expa = ishell.exp(a) #exponents
                expb = jshell.exp(b)
                coefa = ishell.coef(a)  #coefficients
                coefb = jshell.coef(b)
                AMa = ishell.am  #angular momenta
                AMb = jshell.am
                #defining centers for each basis function 
                #mol.x() returns the x coordinate of the atom given by ishell.ncenter
                #we use this to define a coordinate vector for our centers
                A = [mol.x(ishell.ncenter), mol.y(ishell.ncenter), mol.z(ishell.ncenter)]
                B = [mol.x(jshell.ncenter), mol.y(jshell.ncenter), mol.z(jshell.ncenter)]
~~~

We are now ready to define all the parameters that go into our recursion function and define the initial starting point `start` of our recursion which should be the $s$ orbital overlap:

\begin{equation} 
s = (\mathbf 0_A | \mathbf 0_B) = \left(\frac{\pi}{\alpha}\right)^{3/2} \exp[-\zeta(\mathbf A-\mathbf B)^2] 
\end{equation}
                
                
~~~python
                alpha = expa + expb
                zeta = (expa * expb) / alpha
                P = (expa * A + expb * B) / alpha
                PA = P - A
                PB = P - B
                AB = A - B
                start = (np.pi / alpha)**(3 / 2) * np.exp(-zeta * (AB[0]**2 + AB[1]**2 + AB[2]**2))
                # call the recursion function
                x, y, z = os_recursion(PA, PB, alpha, AMa, AMb, start)
~~~


We are now ready to assign the elements of our overlap matrix. Currently, we are only looping over the shells, but not the individual components of each shell (e.g. we are not distinguishing between $p_x$, $p_y$, and $p_z$). Given the total angular momentum of *a* `AMa` and *b* `AMb`, we loop over all possible allocations of this total angular momentum to each angular momentum index in $\mathbf{n} = (n_x, n_y, n_z)$ which we denote as l, m, and n.

~~~python
                # Basis function index where the shell begins
                i_idx = ishell.function_index  
                j_idx = jshell.function_index
                
                # We use counters to keep track of which component (e.g., p_x, p_y, p_z)
                # within the shell we are on
                counta = 0
                
                for p in range(AMa + 1):
                    la = AMa - p                    # Let l take on all values, and p be the leftover a.m.
                    for q in range(p + 1):
                        ma = p - q                  # distribute all leftover a.m. to m and n
                        na = q
                        countb = 0
                        for r in range(AMb + 1):
                            lb = AMb - r            # Let l take on all values, and r the leftover a.m.
                            for s in range(r + 1):
                                mb = r - s          # distribute all leftover a.m. to m and n
                                nb = s
                                
                                
                                S[i_idx + counta, j_idx + countb] += start    \
                                                                   * coefa    \
                                                                   * coefb    \
                                                                   * x[la,lb] \
                                                                   * y[ma,mb] \
                                                                   * z[na,nb] \
 
                                countb += 1
                        counta += 1
~~~

## Computing the Full Overlap Integral Matrix

Putting all the above code snippets together we form our full overlap matrix:

In [8]:
# make space to store the overlap integral matrix
S = np.zeros((basis.nao(),basis.nao()))

# loop over the shells, basis.nshell is the number of shells
for i in range(basis.nshell()):
    for j in range(basis.nshell()):
        #basis.shell is a shell (1s, 2s, 2p, etc.)
        #for water, there are 5 shells: (H: 1s, H: 1s, O: 1s, 2s, 2p)
        ishell = basis.shell(i) 
        jshell = basis.shell(j)
        #each shell has some number of primitives which make up each component of a shell
        #sto-3g has 3 primitives for every component of every shell.
        nprimi = ishell.nprimitive 
        nprimj = jshell.nprimitive
        #loop over the primitives within a shell
        for a in range(nprimi):  
            for b in range(nprimj):
                expa = ishell.exp(a) #exponents
                expb = jshell.exp(b)
                coefa = ishell.coef(a)  #coefficients
                coefb = jshell.coef(b)
                AMa = ishell.am  #angular momenta
                AMb = jshell.am
                #defining centers for each basis function 
                #mol.x() returns the x coordinate of the atom given by ishell.ncenter
                #we use this to define a coordinate vector for our centers
                A = np.array([mol.x(ishell.ncenter), mol.y(ishell.ncenter), mol.z(ishell.ncenter)])
                B = np.array([mol.x(jshell.ncenter), mol.y(jshell.ncenter), mol.z(jshell.ncenter)])

                alpha = expa + expb
                zeta = (expa * expb) / alpha
                P = (expa * A + expb * B) / alpha
                PA = P - A
                PB = P - B
                AB = A - B
                start = (np.pi / alpha)**(3 / 2) * np.exp(-zeta * (AB[0]**2 + AB[1]**2 + AB[2]**2))
               
                # call the recursion
                x, y, z = os_recursion(PA, PB, alpha, AMa+1, AMb+1)

                
                # Basis function index where the shell begins
                i_idx = ishell.function_index  
                j_idx = jshell.function_index
                
                # We use counters to keep track of which component (e.g., p_x, p_y, p_z)
                # within the shell we are on
                counta = 0
                
                for p in range(AMa + 1):
                    la = AMa - p                    # Let l take on all values, and p be the leftover a.m.
                    for q in range(p + 1):
                        ma = p - q                  # distribute all leftover a.m. to m and n
                        na = q
                        countb = 0
                        for r in range(AMb + 1):
                            lb = AMb - r            # Let l take on all values, and r the leftover a.m.
                            for s in range(r + 1):
                                mb = r - s          # distribute all leftover a.m. to m and n
                                nb = s
                                
                                
                                S[i_idx + counta, j_idx + countb] += start    \
                                                                   * coefa    \
                                                                   * coefb    \
                                                                   * x[la,lb] \
                                                                   * y[ma,mb] \
                                                                   * z[na,nb] \
 
                                countb += 1
                        counta += 1

Check our overlap against Psi4:

In [3]:
mints = psi4.core.MintsHelper(basis)
Spsi4 = np.asarray(mints.ao_overlap())
np.allclose(S, Spsi4, 6)

NameError: name 'psi4' is not defined


# Kinetic energy integrals 
Before proceeding to an analysis of the kinetic energy integral, it will prove convenient to establish a short hard notation for integrals related to the overlap integral:
\begin{equation}
(0|0) = \int G(\alpha_a, \mathbf A, l_a, m_a, n_a) G(\alpha_b, \mathbf B, l_b, m_b, n_b)\ d\tau
\end{equation}
The symbol $(\mathrm{+n}  |  0)_{\mathrm{x}}$ will denote an integral of the form given by the above equation, except that the quantum number $l_a$ has been incremented by n. Similar notation applies to the $m_a$ and $n_a$ quantum numbers with the use of subscripts y and z.  We will also use the symbols $(0  |+\mathrm{n}  )_{\mathrm{x}}$ , where we have incremented the quantum number $l_b$ by n.

The kinetic energy integral is defined as

\begin{equation}
T = - \frac{1}{2}  \int G(\alpha_a, \mathbf A, l_a, m_a, n_a) \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) G(\alpha_b, \mathbf B, l_b, m_b, n_b)\ d\tau
\end{equation}

Evaluate the kinetic energy integrals for $I_x$, $I_y$, and $I_z$ and code them up to obtain $T_{ij}$. The equations that you obtain for these integrals are "unsymmetric" because the quantum numbers of the Gaussian-type function centered on $\mathbf B$ are altered while those of the Gaussian-type function on $\mathbf A$ are not. The symmetric x, y, and z components are given by:


\begin{equation}
T_x = \frac{1}{2}\Big\{l_a l_b (-1|-1)_{\mathrm{x}} + 4\alpha_a \alpha_b (+1|+1)_{\mathrm{x}} - 2\alpha_a l_b (+1|-1)_{\mathrm{x}} - 2\alpha_b l_a (-1|+1)_{\mathrm{x}}\Big\}
\end{equation}

\begin{equation}
T_y = \frac{1}{2}\Big\{l_a l_b (-1|-1)_{\mathrm{y}} + 4\alpha_a \alpha_b (+1|+1)_{\mathrm{y}} - 2\alpha_a l_b (+1|-1)_{\mathrm{y}} - 2\alpha_b l_a (-1|+1)_{\mathrm{y}}\Big\}
\end{equation}

\begin{equation}
T_z = \frac{1}{2}\Big\{l_a l_b (-1|-1)_{\mathrm{z}} + 4\alpha_a \alpha_b (+1|+1)_{\mathrm{z}} - 2\alpha_a l_b (+1|-1)_{\mathrm{z}} - 2\alpha_b l_a (-1|+1)_{\mathrm{z}}\Big\}
\end{equation}

and the full kinetic energy integral is:

\begin{equation}
T = c_a c_b (T_x + T_y + T_z)
\end{equation}

where $c_a$ and $c_b$ are the coefficients of the primitives **a** and **b**.


Within our same for loop structure as for the overlap integrals, we can compute the kinetic energy integrals with:

~~~python
Tx = (1 / 2) * (la * lb * x[la - 1, lb - 1] + 4 * expa * expb * x[la + 1, lb + 1] \
                - 2 * expa * lb * x[la + 1, lb - 1] - 2 * expb * la * x[la - 1, lb + 1]) \
                * y[ma, mb] * z[na, nb]
                
Ty = (1 / 2) * (ma * mb * y[ma - 1, mb - 1] + 4 * expa * expb * y[ma + 1, mb + 1] \
                - 2 * expa * mb * y[ma + 1, mb - 1] - 2 * expb * ma * y[ma - 1, mb + 1]) \
                * x[la, lb] * z[na, nb]

Tz = (1 / 2) * (na * nb * y[na - 1, nb - 1] + 4 * expa * expb * y[na + 1, nb + 1] \
                - 2 * expa * nb * y[na + 1, nb - 1] - 2 * expb * na * y[na - 1, nb + 1]) \
                * x[la, lb] * y[ma, mb]

T[i_idx + counta, j_idx + countb] += start * coefa * coefb * (Tx + Ty + Tz)
~~~

# Dipole moment integrals
We discuss these non-energy integrals here because they are frequently used and closely connected with the overlap and kinetic energy integrals. The dipole moment is defined with respect to a point in space $\mathbf C$, which is almost always taken to be the center of mass. Fortunately for our purposes, Psi4 automatically moves the molecule to be centered on the center of mass and thus $\mathbf C$ is $\mathbf 0$. The dipole moment integral is written for the $x$ direction as
\begin{equation}
d_{\mathrm{x}} =  \int G(\alpha_a, \mathbf A, l_a, m_a, n_a) G(\alpha_b, \mathbf B, l_b, m_b, n_b) \mathrm{x_c}\ d\tau
\end{equation}
and similarly for the operators $\mathrm{y_c}$ and $\mathrm{z_c}$. A convenient procedure is to redefine $\mathrm{x_c}$ in terms of  $\mathrm{x_A}$ or $\mathrm{x_B}$, and we will use $\mathrm{x_A}$. Thus
\begin{equation}
\mathrm{x_c} = \mathrm{x} - \mathrm{C_x} = (\mathrm{x - A_x}) + (\mathrm{A_x - C_x}) = \mathrm{x_A + \mathbf {AC}_x}
\end{equation}
and we find
\begin{equation}
d_\mathrm{x} = (+1|0)_{\mathrm{x}} + \mathbf{AC}_{\mathrm{x}} (0|0)
\end{equation}
Similarily for $d_{\mathrm{y}}$ and $d_{\mathrm{z}}$.


~~~python
dx = (x[la + 1, lb] + A[0] * x[la, lb]) * y[ma, mb] * z[na, nb]
dy = (y[ma + 1, mb] + A[1] * y[ma, mb]) * x[la, lb] * z[na, nb]
dz = (z[na + 1, nb] + A[2] * z[na, nb]) * x[la, lb] * y[ma, mb]

Dx[i_idx + counta, j_idx + countb] += start * coefa * coefb * dx
Dy[i_idx + counta, j_idx + countb] += start * coefa * coefb * dy
Dz[i_idx + counta, j_idx + countb] += start * coefa * coefb * dz
~~~

Putting everything together, we get the overlap, kinetic, and dipole integrals:

In [5]:
# make space to store the overlap integral matrix
S = np.zeros((basis.nao(),basis.nao()))
T = np.zeros((basis.nao(),basis.nao()))
Dx = np.zeros((basis.nao(),basis.nao()))
Dy = np.zeros((basis.nao(),basis.nao()))
Dz = np.zeros((basis.nao(),basis.nao()))

# loop over the shells, basis.nshell is the number of shells
for i in range(basis.nshell()):
    for j in range(basis.nshell()):
        #basis.shell is a shell (1s, 2s, 2p, etc.)
        #for water, there are 5 shells: (H: 1s, H: 1s, O: 1s, 2s, 2p)
        ishell = basis.shell(i) 
        jshell = basis.shell(j)
        #each shell has some number of primitives which make up each component of a shell
        #sto-3g has 3 primitives for every component of every shell.
        nprimi = ishell.nprimitive 
        nprimj = jshell.nprimitive
        #loop over the primitives within a shell
        for a in range(nprimi):  
            for b in range(nprimj):
                expa = ishell.exp(a) #exponents
                expb = jshell.exp(b)
                coefa = ishell.coef(a)  #coefficients
                coefb = jshell.coef(b)
                AMa = ishell.am  #angular momenta
                AMb = jshell.am
                #defining centers for each basis function 
                #mol.x() returns the x coordinate of the atom given by ishell.ncenter
                #we use this to define a coordinate vector for our centers
                A = np.array([mol.x(ishell.ncenter), mol.y(ishell.ncenter), mol.z(ishell.ncenter)])
                B = np.array([mol.x(jshell.ncenter), mol.y(jshell.ncenter), mol.z(jshell.ncenter)])

                alpha = expa + expb
                zeta = (expa * expb) / alpha
                P = (expa * A + expb * B) / alpha
                PA = P - A
                PB = P - B
                AB = A - B
                start = (np.pi / alpha)**(3 / 2) * np.exp(-zeta * (AB[0]**2 + AB[1]**2 + AB[2]**2))
               
                # call the recursion
                x, y, z = os_recursion(PA, PB, alpha, AMa+1, AMb+1)

                
                # Basis function index where the shell begins
                i_idx = ishell.function_index  
                j_idx = jshell.function_index
                
                # We use counters to keep track of which component (e.g., p_x, p_y, p_z)
                # within the shell we are on
                counta = 0
                
                for p in range(AMa + 1):
                    la = AMa - p                    # Let l take on all values, and p be the leftover a.m.
                    for q in range(p + 1):
                        ma = p - q                  # distribute all leftover a.m. to m and n
                        na = q
                        countb = 0
                        for r in range(AMb + 1):
                            lb = AMb - r            # Let l take on all values, and r the leftover a.m.
                            for s in range(r + 1):
                                mb = r - s          # distribute all leftover a.m. to m and n
                                nb = s
                                S[i_idx + counta, j_idx + countb] += start    \
                                                                   * coefa    \
                                                                   * coefb    \
                                                                   * x[la,lb] \
                                                                   * y[ma,mb] \
                                                                   * z[na,nb] \
                                Tx = (1 / 2) * (la * lb * x[la - 1, lb - 1] + 4 * expa * expb * x[la + 1, lb + 1] \
                                       - 2 * expa * lb * x[la + 1, lb - 1] - 2 * expb * la * x[la - 1, lb + 1])   \
                                       * y[ma, mb] * z[na, nb]

                                Ty = (1 / 2) * (ma * mb * y[ma - 1, mb - 1] + 4 * expa * expb * y[ma + 1, mb + 1] \
                                       - 2 * expa * mb * y[ma + 1, mb - 1] - 2 * expb * ma * y[ma - 1, mb + 1])   \
                                       * x[la, lb] * z[na, nb]

                                Tz = (1 / 2) * (na * nb * y[na - 1, nb - 1] + 4 * expa * expb * y[na + 1, nb + 1] \
                                       - 2 * expa * nb * y[na + 1, nb - 1] - 2 * expb * na * y[na - 1, nb + 1])   \
                                       * x[la, lb] * y[ma, mb]

                                T[i_idx + counta, j_idx + countb] += start * coefa * coefb * (Tx + Ty + Tz)


                                dx = (x[la + 1, lb] + A[0] * x[la, lb]) * y[ma, mb] * z[na, nb]
                                dy = (y[ma + 1, mb] + A[1] * y[ma, mb]) * x[la, lb] * z[na, nb]
                                dz = (z[na + 1, nb] + A[2] * z[na, nb]) * x[la, lb] * y[ma, mb]

                                Dx[i_idx + counta, j_idx + countb] += start * coefa * coefb * dx
                                Dy[i_idx + counta, j_idx + countb] += start * coefa * coefb * dy
                                Dz[i_idx + counta, j_idx + countb] += start * coefa * coefb * dz
                                
                                countb += 1
                        counta += 1

SyntaxError: invalid syntax (<ipython-input-5-7721baa75b26>, line 65)

Check the kinetic energy integrals:

In [ ]:
Tpsi4 = np.asarray(mints.ao_kinetic())
np.allclose(T, Tpsi4, 6)


# Computing the Dipole Moment

#### The x component
\begin{align}
\mu_x =& - \mu_{\mathrm{elec}} + \mu_{\mathrm{nuclear}}\\
\mu_x =& - \sum_{\mu \nu}^{\mathrm{AO}} D_{\mu \nu} d_{\mu \nu}^{x} + \sum_A^N Z_A X_A
\end{align}

You can obtain the nuclear contribution to the dipole $\mu_{\mathrm{nuclear}}$ from Psi4 using

    psi4.core.molecule.nuclear_dipole()

#### The total dipole moment
\begin{equation}
\mu = \sqrt{\mu_x^2 + \mu_y^2 + \mu_z^2}
\end{equation}

In [6]:
#Code to compute dipole moment
D = np.asarray(scf_wfn.Da())

nuc_dipole = np.asarray(mol.nuclear_dipole())

mux = -np.einsum('pq,pq->', D, Dx) + nuc_dipole[0]
muy = -np.einsum('pq,pq->', D, Dy) + nuc_dipole[1]
muz = -np.einsum('pq,pq->', D, Dz) + nuc_dipole[2]

mu = np.sqrt(mux**2 + muy**2 + muz**2)
print(mu)

psi4mu = oeprop(wfn, 'DIPOLE', title='H3O+ SCF')
print(psi4mu)